In [124]:
# Split index data into 24 intervals 
    # For continous train-validate-test split
    # Save split index data in 'data/pickled' folder

# Imports (External)
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from monthdelta import monthdelta

import xlrd
import xlsxwriter
from collections import OrderedDict
import pickle

import sys
sys.path.append('../')  
# Internal Imports
from wsae_lstm.utils import dictmap_load
#from wsae_lstm.utils import interval_split,dict_interval_split
from wsae_lstm.utils import pickle_save


In [125]:
# Load clean data from data/processed folder
dict_dataframes_index = dictmap_load(path = "../data/processed/clean_data_index.xlsx")  
#print(dict_dataframes_index.keys())

In [126]:
dict_dataframes_index['csi300 index data'].head(2)

,date,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
0,2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
1,2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08


In [127]:
dict_dataframes_index['csi300 index data'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 20 columns):
date               2010 non-null datetime64[ns]
open               2010 non-null float64
high               2010 non-null float64
low                2010 non-null float64
close              2010 non-null float64
volume             2010 non-null float64
us dollar index    2010 non-null float64
shibor             2010 non-null float64
macd               2010 non-null float64
cci                2010 non-null float64
atr                2010 non-null float64
boll               2010 non-null float64
ema20              2010 non-null float64
ma10               2010 non-null float64
mtm6               2010 non-null float64
ma5                2010 non-null float64
mtm12              2010 non-null float64
roc                2010 non-null float64
smi                2010 non-null float64
wvad               2010 non-null float64
dtypes: datetime64[ns](1), float64(19)
memory usage: 314.1 KB


In [128]:
def interval_split(df):
    """Split dataframe contents into train/validate/test intervals as defined
    in Bao, Yue, Rao (2017): 24 interval split."""
    dict_dataframes = {}
    split_count = 24
    month_increment = 0
    interval_index = 1
    
    df = df.set_index(df['date'])
    while split_count > 0:
        front = df['date'][0] 
        front = front + monthdelta(month_increment)
        back = df['date'][0]+monthdelta(30) 
        back = back + monthdelta(month_increment)
        month_increment += 3
        split_count -= 1
        df_interval = pd.DataFrame(df[(df['date'] >= front) & (df['date'] <= back)])
        dict_dataframes[interval_index] = df_interval
        #############################
        # drop date column - already set as datetimeindex param
        # without this there is double index with index set + date column
        dict_dataframes[interval_index].drop(['date'],axis=1,inplace=True)
        #############################
        interval_index += 1
        #print(front,back)
    

    return dict_dataframes
def dict_interval_split(dict_dataframes):
    """Apply 24-interval split to dictionary of dataframes; 
    interval_split() function applied to each dataframe in dict object. """
    subdict_dataframes = {}
    for dataframe in dict_dataframes:
        subdict_dataframes[dataframe] = interval_split(dict_dataframes[dataframe])

    return subdict_dataframes

In [129]:
#dict_dataframes_index['csi300 index data'][1].drop(['date'],axis=1,inplace=True)

In [130]:
# Split each dataframe into 24 intervals/periods for train/validate/test split
dict_df_interval = dict_interval_split(dict_dataframes_index)
print(dict_df_interval.keys())



dict_keys(['csi300 index data', 'nifty 50 index data', 'hangseng index data', 'nikkei 225 index data', 's&p500 index data', 'djia index data'])


In [131]:
dict_df_interval['csi300 index data'][3].head()

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2009-01-05,1848.33,1882.96,1837.84,1882.96,481870.20,82.55,0.9086,-19.737316,-57.712606,65.237,1947.478077,1914.604542,1899.7427,-4.116,1850.1938,-111.491,-5.590062,-0.082430,1.654028e+08
2009-01-06,1880.67,1948.49,1873.01,1942.80,709740.97,82.78,0.8967,-14.716425,-4.275006,75.478,1950.259692,1917.289348,1888.8108,72.030,1866.3336,-58.625,-2.929170,-0.037052,1.934664e+08
2009-01-07,1942.67,1959.25,1930.87,1931.18,597021.67,82.17,0.8894,-11.541682,16.983195,28.378,1954.154077,1918.612076,1880.1737,69.082,1881.6178,-113.924,-5.570578,0.006329,1.313906e+08
2009-01-08,1894.66,1902.80,1873.65,1887.99,534464.90,81.58,0.8829,-12.367935,-29.633141,57.530,1955.068923,1915.695783,1877.0773,33.234,1892.5290,-164.123,-7.997753,0.000485,3.413567e+07
2009-01-09,1886.49,1923.42,1886.49,1918.37,474072.16,82.52,0.8363,-10.451343,3.370722,36.935,1956.981769,1915.949995,1880.2063,84.930,1912.6576,-99.184,-4.916064,0.024813,7.071998e+07


In [132]:
dict_df_interval['csi300 index data'][1].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 612 entries, 2008-07-01 to 2010-12-31
Data columns (total 19 columns):
open               612 non-null float64
high               612 non-null float64
low                612 non-null float64
close              612 non-null float64
volume             612 non-null float64
us dollar index    612 non-null float64
shibor             612 non-null float64
macd               612 non-null float64
cci                612 non-null float64
atr                612 non-null float64
boll               612 non-null float64
ema20              612 non-null float64
ma10               612 non-null float64
mtm6               612 non-null float64
ma5                612 non-null float64
mtm12              612 non-null float64
roc                612 non-null float64
smi                612 non-null float64
wvad               612 non-null float64
dtypes: float64(19)
memory usage: 95.6 KB


In [133]:
# Save new dict of dataframes with interval splits to disk (using pickle)
pickle_save(dict_df_interval,"../data/pickled/clean_data_index_interval")